In [ ]:
from flask import Flask, request, abort

import actions
import auth
import flask
import pandas as pd
import ast
import json

import threading
import time

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *


# Create Flask object called app.
app = Flask(__name__)


# Create root to easily let us know its on/working.
@app.route('/')
def root():
    return 'online'


@app.route('/webhook', methods=['POST'])
def webhook():
    if request.method == 'POST':
        try:
            global a
            global order_status
            global hung_order
            global sent_order
            global last_order_closed
            
            the_list = ['PendingSubmit',
                        'PendingCancel',
                        'PreSubmitted', 
                        'Submitted', 
                        'ApiCancelled', 
                        'ApiPending']

            tester = str(request.get_data())
            
            ### Creates a byte object instead of a dict from the literal eval 
            ### This is an issue with TradingView's alert data types ###
            data = str(ast.literal_eval(str(tester)))
            
            
            ##############################################
            ### Parses out data from TradingView alert ###
            ##############################################
            
            sep = '@'
            literal = data.split(sep, 1)[0]

            sep_2 = '$'
            literal = literal.split(sep_2, 1)[1]
            literal = ast.literal_eval(str(literal))

            type_purchase = data.split(sep, 1)[1]

            sell = type_purchase.find('sell')
            buy = type_purchase.find('buy')
            closing = type_purchase.find('closing')
            initiator = type_purchase.find('initiate')
            closer = type_purchase.find('2')
            ##############################################
            
            
            
            ###################################################
            ### Initiates variables and  allocates buy types###
            ###################################################
            
            if initiator == 0:
                a = 0
                order_status = 0
                hung_order = 0
                last_order_closed = 0
            
            if buy >= 0:
                literal['type'] = 'BUY'
                if (closing == 0) or (closer > 0):
                    print('CLOSING ORDER')
                    a = a+1
                    print('the a count is: ', a)

            else: 
                if sell > 0:
                    literal['type'] = 'SELL'
                    if (closing == 0) or (closer > 0):
                        print('CLOSING ORDER') 
                        a = a+1
                        print('the a count is: ', a)
                        
            print('last order closed = ', last_order_closed)
                
            print(data, buy, sell, closing,initiator)
            data = literal
            ###################################################
            
            
            ##################################################
            ########## Interactive Brokers Functions #########
            ##################################################

            class IBapi(EWrapper, EClient):
                def __init__(self):
                    EClient.__init__(self, self)

                def nextValidId(self, orderId: int):
                    super().nextValidId(orderId)
                    self.nextorderId = orderId
                    print('The next valid order id is: ', self.nextorderId)
                    
                def openOrder(self, orderId: int, contract: float, order: float,orderState: str):
                    super().openOrder(orderId, contract, order, orderState)
                    self.fill_status = orderState.status
                    print("OpenOrder. PermId: ", order.permId, "ClientId:", order.clientId, " OrderId:", orderId, 
                   "Account:", order.account, "Symbol:", contract.symbol, "SecType:", contract.secType,
                   "Exchange:", contract.exchange, "Action:", order.action, "OrderType:", order.orderType,
                   "TotalQty:", order.totalQuantity, "CashQty:", order.cashQty, 
                   "LmtPrice:", order.lmtPrice, "AuxPrice:", order.auxPrice, "Status:", orderState.status)
     
                    order.contract = contract


            def run_loop():
                app_ib.run()

            #Function to create FX Order contract
            def contract_order(symbol):
                contract = Contract()
                contract.symbol = symbol
                contract.secType = 'FUT'
                contract.exchange = 'GLOBEX'
                contract.currency = 'USD'
                contract.lastTradeDateOrContractMonth  = '202009'
                return contract
            
            ##################################################

            
            ##################################################
            ########## Establishes Connections ###############
            ##################################################            
            
            app_ib = IBapi()
            app_ib.connect('127.0.0.1', 7497, 123)

            app_ib.nextorderId = None
            app_ib.fill_status = None

            ### Start the socket in a thread ###
            
            api_thread = threading.Thread(target=run_loop, daemon=True)
            api_thread.start()
            
            #########################################################      
            ###### Check if the API is connected via orderid ########
            #########################################################

            while True:
                if isinstance(app_ib.nextorderId, int):
                    print('')
                    print('good connection to account...')
                    print('')
                    break
                    
                    
            #########################################################      
            ### Checks the order queue for any outstanding orders ###
            ###              Begins Logic Looping                 ###
            #########################################################
            
            ### Checks queue for order status and hung orders in queue ###
            
            if (app_ib.fill_status in the_list)&(a<3):
                print('')
                print('THIS IS THE CURRENT QUEUE ORDER STATUS: ', app_ib.fill_status)
                print('')
                order_status = 0
                hung_order = hung_order + 1
                
                ### If a closing order, add 2 to a ###
                if closer < 0:
                    if closing < 0:
                        print('adding 2 to a')
                        a = a + 2
                    
                ### If the order is hung in the queue, then cancel the order coming in ###
                ### Note: This only applies to orders that are entering orders, not closing ###
                if (hung_order >= 1) & (hung_order <2):
                    print('THIS IS A HUNG ORDER')
                    hung_order = hung_order + 1
                    print('hung order number: ', hung_order, 'the a count is: ', a)
    
                    ### Cancels all orders if an entering order is hung ###
                    if (hung_order >= 2) & (a <= 1):
                        print('cancelling all orders')
                        hung_order = 0
                        
                        app_ib.reqGlobalCancel()
                        app_ib.disconnect()
                        
            ### Sets the order status at 0 ###
            if initiator >= 0:
                order_status = 0
                print('model initiated')
           
        
            elif(app_ib.fill_status not in the_list):
                
                ### If the order is coming off of a hung status, only allows reset orders (non-closing orders) ###
                if (((closing >= 0)&(a>1)) | ((last_order_closed == 1) & ((closing >= 0)|(closer >=0)))):
                    print('Waiting for reset order..')
                    order_status = 0
                else:
                    print('NO ORDER IN QUEUE')
                    order_status = 1

            ### If a closing order is hung, cancels all incoming orders ###
            elif(hung_order >= 2) & (a > 1):
                if (app_ib.fill_status in the_list):
                    print('cancelling current order #: ', app_ib.nextorderId)
                    app_ib.cancelOrder(app_ib.nextorderId)

            ### If all gates are passed, allows closing orders through ###
            elif (closing >= 0) and (a < 1):
                print('')
                print('THIS IS A CLOSING SALE')
                order_status = 1
                print('The a count is :', a)
                print('second')
                
    
            ### Sends an order if all previous criteria are met ###
            if (order_status == 1)&(initiator < 0):
                
                #Creates order object
                order = Order()
                ticker = data['ticker']
                contract_expr = '202009' ########### This will change as contract expiration changes ###########
                order.action = data['type']
                order.totalQuantity = 1
                order.orderType = 'LMT'
                order.lmtPrice = str(data['price'])

                #Places order
                app_ib.placeOrder(app_ib.nextorderId, contract_order(ticker), order)
                app_ib.nextorderId += 1
                print('Order sent: ', order.orderType, '', order.action, '','on ', ticker, '', contract_expr, '@ ', order.lmtPrice)
                print('')
                print('#########################################')
                print('###### YOU ARE IN A', data['type'].upper(),'POSITION ######')
                print('#########################################')
                hung_order = 0
                time.sleep(1)
                
                
                print('resetting a')
                a = 0
                
                if (closing >= 0) |(closer > 0):
                    last_order_closed = 1
                else: 
                    last_order_closed = 0
                    
                print('the a count is: ', a)
                app_ib.disconnect()

      
            else: 
                print('Still waiting for previous order to process...')
                print('')
                print('    ONLY CLOSING OR RESET ORDERS ALLOWED      ')
                print('the a count is: ', a)
                app_ib.disconnect()

        ######################################### 
        ######################################### 

        except:
            traceback.print_exc(file=sys.stdout)     
        return '', 200
    
if __name__ == '__main__':
    app.run()
            

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
127.0.0.1 - - [13/Jul/2020 23:47:18] "POST /webhook HTTP/1.1" 200 -
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


last order closed =  0
b'${"ticker":"ES", "price":3000}@initiate!1' -1 -1 -1 0
The next valid order id is:  584

good connection to account...

model initiated
Still waiting for previous order to process...

    ONLY CLOSING OR RESET ORDERS ALLOWED      
the a count is:  0


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


last order closed =  0
b'${"ticker":"ES", "price":3151}@short_willr_1 sell! 1' -1 14 -1 -1
The next valid order id is:  584

good connection to account...

NO ORDER IN QUEUE
Order sent:  LMT  SELL  on  ES  202009 @  3151

#########################################
###### YOU ARE IN A SELL POSITION ######
#########################################
OpenOrder. PermId:  459075941 ClientId: 123  OrderId: 584 Account: DU2258209 Symbol: ES SecType: FUT Exchange: GLOBEX Action: SELL OrderType: LMT TotalQty: 1.0 CashQty: 0.0 LmtPrice: 3151.0 AuxPrice: 0.0 Status: Submitted


...

The next valid order id is:  592

good connection to account...

NO ORDER IN QUEUE
Order sent:  LMT  BUY  on  ES  202009 @  3151

#########################################
###### YOU ARE IN A BUY POSITION ######
#########################################


ERROR -1 2158 Sec-def data farm connection is OK:secdefil


OpenOrder. PermId:  459075950 ClientId: 123  OrderId: 592 Account: DU2258209 Symbol: ES SecType: FUT Exchange: GLOBEX Action: BUY OrderType: LMT TotalQty: 1.0 CashQty: 0.0 LmtPrice: 3151.0 AuxPrice: 0.0 Status: PreSubmitted
OpenOrder. PermId:  459075950 ClientId: 123  OrderId: 592 Account: DU2258209 Symbol: ES SecType: FUT Exchange: GLOBEX Action: BUY OrderType: LMT TotalQty: 1.0 CashQty: 0.0 LmtPrice: 3151.0 AuxPrice: 0.0 Status: Filled
OpenOrder. PermId:  459075950 ClientId: 123  OrderId: 592 Account: DU2258209 Symbol: ES SecType: FUT Exchange: GLOBEX Action: BUY OrderType: LMT TotalQty: 1.0 CashQty: 0.0 LmtPrice: 3151.0 AuxPrice: 0.0 Status: Filled


127.0.0.1 - - [14/Jul/2020 01:08:02] "POST /webhook HTTP/1.1" 200 -


resetting a
the a count is:  0


In [ ]:
    if request.method == 'POST':
        try:           
            #data.to_dict()longbuy
            #data = json.loads(request.get_data(as_text=True))
            #data = ast.literal_eval(request.get_data(as_text=True))
            
            #data = {'price':'3200', 'ticker':'RTY', 'type':'buy'}
            ####### Defines IB functions to utilize #######

            class IBapi(EWrapper, EClient):
                def __init__(self):
                    EClient.__init__(self, self)

                def nextValidId(self, orderId: int):
                    super().nextValidId(orderId)
                    self.nextorderId = orderId
                    print('The next valid order id is: ', self.nextorderId)

            def run_loop():
                app_ib.run()

            #Function to create FX Order contract
            def contract_order(symbol):
                contract = Contract()
                contract.symbol = symbol
                contract.secType = 'FUT'
                contract.exchange = 'GLOBEX'
                contract.currency = 'USD'
                contract.lastTradeDateOrContractMonth  = '202009'
                return contract

            ######################################  
            ########## Sends BUY order  ##########
            ######################################  
            
            app_ib = IBapi()
            app_ib.connect('127.0.0.1', 7497, 123)

            app_ib.nextorderId = None
            print(app_ib.nextorderId)
            #Start the socket in a thread
            api_thread = threading.Thread(target=run_loop, daemon=True)
            api_thread.start()

            #Check if the API is connected via orderid
            while True:
                if isinstance(app_ib.nextorderId, int):
                    print('')
                    print('good connection to account...')
                    print('')
                    break
                else:
                    print('waiting for connection')
                    time.sleep(1)

            #Create order object
            order = Order()
            ticker = 'ES'
            contract_expr = '202009' ########### This will change as contract expiration changes ###########
            order.action = 'buy'
            order.totalQuantity = 1
            order.orderType = 'LMT'
            order.lmtPrice = '3088'

            #Place order
            app_ib.placeOrder(app_ib.nextorderId, contract_order(ticker), order)
            app_ib.nextorderId += 1
            print('Order sent: ', order.orderType, '', order.action, '','on ', ticker, '', contract_expr, '@ ', order.lmtPrice)
            print('')
            print('################################################')
            print('###### YOU HAVE PURCHASED 100 LIVE CATTLE ######')
            print('################################################')
            app_ib.disconnect()

            ######################################### 
            ######################################### 

In [ ]:
the_list = ['PendingSubmit',
'PendingCancel',
'PreSubmitted', 
'Submitted', 
'ApiCancelled', 
'ApiPending']


In [ ]:
## IB Order Status Strings
  * **PendingSubmit**: indicates that you have transmitted the order, but have not yet received confirmation that it has been accepted by the order destination. 
  * **PendingCancel**: indicates that you have sent a request to cancel the order but have not yet received cancel confirmation from the order destination. At this point, your order is not confirmed canceled. It is not guaranteed that the cancellation will be successful. 
  * **PreSubmitted**: indicates that a simulated order type has been accepted by the IB system and that this order has yet to be elected. The order is held in the IB system until the election criteria are met. At that time the order is transmitted to the order destination as specified . 
  * **Submitted**: indicates that your order has been accepted by the system. 
  * **ApiCancelled**: after an order has been submitted and before it has been acknowledged, an API client client can request its cancelation, producing this state. 
  * **ApiPending**: undocumented; heard it exists from the e-mail group.
  * **Cancelled**: indicates that the balance of your order has been confirmed canceled by the IB system. This could occur unexpectedly when IB or the destination has rejected your order. 
  * **Filled**: indicates that the order has been completely filled. Market orders executions will not always trigger a Filled status. 
  * **Inactive**: indicates that the order was received by the system but is no longer active because it was rejected or canceled.
